In [27]:
using Plots
using JLD
using StatsPlots
using LaTeXStrings

include("../src/FCSeqTools.jl");
include("../src/functions.jl");

folder_name = "EAA_vs_(pseudo_count=0.5__init_pseudo_count=0.01)"

rm(joinpath("figures/comparison_models",folder_name), force=true, recursive=true)
mkdir(joinpath("figures/comparison_models",folder_name))
path = joinpath("figures/comparison_models",folder_name);

L, q = 96, 21;

# Compare Couplings

In [28]:
method = "cumulative"
fraction = 0.3
stop = 0.9
pseudo_count = 0.5
init_pseudo_count = 0.01

# Choose Notebooks ##################################
nbook1 = "test"
nbook2 = "test2"
######################################################

path1 = "../training/" * method * string(fraction) * "_stop=" * string(stop) * "_reg="*string(pseudo_count) * "_h_ps-count=" * string(init_pseudo_count)*"nbook="*nbook1
#path2 = "../training/" * method * string(fraction) * "_stop=" * string(stop) * "_reg="*string(pseudo_count) * "_h_ps-count=" * string(init_pseudo_count)*"nbook="*nbook2

h1 = JLD.load(path1* "/h.jld")["data"]
Jij1 = JLD.load(path1* "/Jij.jld")["data"];
iteration1 = open(path1* "/iterations.txt", "r")
data1 = parse.(Float64, split(readlines(iteration1)[end], " ")[1:end-1])

# Se è un modello dei nostri #################################################
#h2 = JLD.load(path2* "/h.jld")["data"]
#Jij2 = JLD.load(path2* "/Jij.jld")["data"];
#iteration2 = open(path2* "/iterations.txt", "r");
#data2 = parse.(Float64, split(readlines(iteration2)[end], " ")[1:end-1])

edge_n1, element_n1 = data1[6], data1[7]
#edge_n2, element_n2 = data2[6], data2[7]
#############################################################################

# Se è un modello di Francesco Zamponi ######################################
filepath2 = "../saved_models/Francesco_Zamponi/Parameters_CM_EAA-PCD40.dat";
Jij2, h2 = read_model(filepath2);
#############################################################################

Nij_tot = L * (L - 1)
Nij_ab_tot = Nij_tot * q^2;

#### Scatter Plot of all the Jijs 

In [29]:
n_element_plot = 500_000
Jij1_to_plot = zeros(n_element_plot);
Jij2_to_plot = zeros(n_element_plot);

for i in 1:n_element_plot
    j = rand(1:length(Jij1))
    Jij1_to_plot[i] = Jij1[j]
    Jij2_to_plot[i] = Jij2[j]
end

scatter(Jij1_to_plot, Jij2_to_plot, markershape=:circle, label=:false, xlabel="Jij(a,b) nb.1", ylabel="Jij(a,b) nb.2", title="Couplings nb.1 vs nb.2", alpha=0.2)
savefig(joinpath(path,"All_the_Jij1_vs_Jij2.png"));

### Count Jijs in common and plot the distributions

In [30]:
Jij_plot_1 = Jij1
Jij_plot_2 = Jij2

n_ij, n_ij_ab, n_01, n_10, n_00, Jij_common1, Jij_common2 = count_common_Jij(Jij_plot_1, Jij_plot_2);

frac_ij_tot = round(n_ij/Nij_tot * 100, digits=1)
frac_ijab_tot = round(n_ij_ab/Nij_ab_tot * 100, digits=1)

frac_ij_1 = round(n_ij/edge_n1 * 100, digits=1)
frac_ijab_1 = round(n_ij_ab/element_n1 * 100, digits=1)
#frac_ij_2 = round(n_ij/edge_n2 * 100, digits=1)
#frac_ijab_2 = round(n_ij_ab/element_n2 * 100, digits=1)

println("Common Edges: \n",frac_ij_tot,"% of the total;   ",  frac_ij_1, "% of the first model;   ")#, frac_ij_2, "% of the second model")
println("\nCommon Elements: \n",frac_ijab_tot,"% of the total;   ",  frac_ijab_1, "% of the first model;   ")#, frac_ijab_2, "% of the second model")
println("\nNumber of cases where: J_1 == 0 &  J_2 != 0: ", n_01)
println("\nNumber of cases where: J_1 != 0 &  J_2 == 0: ", n_10)
println("\nNumber of cases where: J_1 == 0 &  J_2 == 0: ", n_00)
println("\nNumber of cases where: J_1 != 0 &  J_2 != 0: ", n_ij_ab)


# Plot Scatter & Distributions #####################################################################################
x, y = Jij_common1[Jij_common1.!= 0.0], Jij_common2[Jij_common2.!= 0.0]
x_label, y_label = L"J_{ij}(a,b), \alpha=0.5", L"J_{ij}(a,b), \alpha=0.05"
title = "Common Couplings: "* L"J_{ij}(a,b)"


plot(  histogram(x, nbins=100, alpha=0.5, orientation=:vertical,label=:false, xticks=[], ylabel="counts", title = title),   
       plot(framestyle = :none),
       scatter(x, y, markershape=:circle, label=:false, xlabel=x_label, ylabel=y_label, alpha=0.2),
       histogram(y, nbins=100, alpha=0.5, orientation=:horizontal, label=:false, yticks=[], xlabel="counts"),
       link = :both, 
       size=(1000, 1000), 
       spacing=0.1, 
       )
savefig(joinpath(path,"Common_Jij1_vs_Jij2.png"));
##################################################################################################################

Common Edges: 
7.1% of the total;   38.6% of the first model;   

Common Elements: 
0.0% of the total;   5.1% of the first model;   

Number of cases where: J_1 == 0 &  J_2 != 0: 35820

Number of cases where: J_1 != 0 &  J_2 == 0: 22576

Number of cases where: J_1 == 0 &  J_2 == 0: 1993681

Number of cases where: J_1 != 0 &  J_2 != 0: 1219


#### Count and plot Jijs NOT in common

In [ ]:
J_diff1 = Jij1[(Jij1.!= 0).*(Jij2.== 0)] 
J_diff2 = Jij2[(Jij2.!= 0).*(Jij1.== 0)]; 

In [ ]:
histogram(J_diff1, bins = 100, title = "Jij non in common", xlabel="values", ylabel="counts", label=:false)
savefig(joinpath(path,"Jij1_NOT_in_common_with_Jij2.png"));

In [ ]:
histogram(J_diff2, bins = 100, title = "Jij non in common", xlabel="values", ylabel="counts", label=:false)
savefig(joinpath(path,"Jij2_NOT_in_common_with_Jij2.png"));

#### Common Jijs vs Score/Iterations


In [ ]:
file_path1 = joinpath(path1, "example_output.txt");
file_path2 = joinpath(path2, "example_output.txt");


edges1, elements1, scores1 = read_example_output(file_path1);
edges2, elements2, scores2 = read_example_output(file_path2);

### Extract Jij's

In [ ]:
n_edges, n_elements, n_edges_x, n_edges_y, n_elements_x, n_elements_y = compare_edges(edges1, elements1, edges2, elements2);

In [ ]:
scatter(scores2, n_elements./n_elements_x * 100, marker=:circle, markersize=3, label="elements")
scatter!(scores2, n_edges./n_edges_x * 100, marker=:circle, markersize=3, label="edges")
plot!(scores2, 90*ones(length(scores2)), label="90%")
plot!(scores2, 50*ones(length(scores2)), label="50%")
xlabel!(L"C_{ij}" * " score")
ylabel!("%")
title!("Percentage of non-zero parameters in both models")
savefig(joinpath(path,"number_shared_Jij_vs_Cij_score.png"));

In [ ]:
scatter(1:length(n_elements), n_elements./n_elements_x * 100, marker=:circle, markersize=3, label="elements")
scatter!(1:length(n_elements), n_edges./n_edges_x * 100, marker=:circle, markersize=3, label="edges")
plot!(1:length(n_elements), 90*ones(length(n_elements)), label="90%")
plot!(1:length(n_elements), 50*ones(length(n_elements)), label="50%")
xlabel!("E.A.A. iteration")
ylabel!("%")
title!("Percentage of non-zero parameters in both models")
savefig(joinpath(path,"number_shared_Jij_vs_iterations.png"));